# OpenAI Agent with Query Engine Tools

## Build Query Engine Tools

In [1]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage

from llama_index.tools import QueryEngineTool, ToolMetadata

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
try:
    storage_context = StorageContext.from_defaults(persist_dir='./storage/lyft')
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir='./storage/uber')
    uber_index = load_index_from_storage(storage_context)
    
    index_loaded = True
except:
    index_loaded = False

In [3]:
if not index_loaded:
    # load data 
    lyft_docs = SimpleDirectoryReader(input_files=["../data/10k/lyft_2021.pdf"]).load_data()
    uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()
    
    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)
    
    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [4]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [5]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine, 
        metadata=ToolMetadata(
            name='lyft_10k', 
            description="Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        )
    ),
    QueryEngineTool(
        query_engine=uber_engine, 
        metadata=ToolMetadata(
            name='uber_10k', 
            description="Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        )
    ),
]


## Setup OpenAI Agent

In [6]:
from llama_index.agent import OpenAIAgent

In [7]:
agent = OpenAIAgent.from_tools(query_engine_tools, verbose=True)

## Let's Try It Out!

In [8]:
response = agent.chat("What was Lyft's revenue growth in 2021?")

=== Calling Function ===
Calling function: lyft_10k with args: {
  "input": "Lyft's revenue growth in 2021"
}
Got output: 
Lyft's revenue growth in 2021 was 36%, driven primarily by the significant increase in the number of Active Riders in 2021 as compared to the prior year, as vaccines became more widely distributed and more communities reopened.


In [23]:
agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  Compare and contrast Lyft and Uber's revenue growth in 2021, then give an analysis


=== Calling Function ===
Calling function: lyft_10k with args: {
  "input": "What was Lyft's revenue growth in 2021?"
}
Got output: 
Lyft's revenue grew by 36% in 2021 compared to the prior year.
=== Calling Function ===
Calling function: uber_10k with args: {
  "input": "What was Uber's revenue growth in 2021?"
}
Got output: 
Uber's revenue growth in 2021 was 57%.
Assistant: In 2021, Lyft's revenue grew by 36% compared to the previous year, while Uber's revenue growth was higher at 57%. This indicates that Uber experienced a faster rate of revenue growth than Lyft in 2021.

The higher revenue growth for Uber can be attributed to several factors. Firstly, Uber has a larger market presence and operates in more countries and cities compared to Lyft. This broader reach allows Uber to capture a larger customer base and generate more revenue.

Secondly, Uber has diversified its business beyond ride-hailing services. The company has expanded into food delivery with Uber Eats, which has exper

Human:  Thanks


Assistant: You're welcome! If you have any more questions, feel free to ask.



Human:  exit
